# Resuspension rate forcing field

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import pickle

%matplotlib inline

#### Parameters:

In [ ]:
# Domain dimensions
imin, imax = 1479, 2179
jmin, jmax = 159, 799

#### Load files:

In [ ]:
# ANHA12 grid
mesh       = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
mesh_lon   = np.array(mesh.variables['nav_lon'])
mesh_lat   = np.array(mesh.variables['nav_lat'])
mesh_bathy = np.array(mesh.variables['hdept'][0])

cond2      = (mesh_bathy > 0.1)
bathy_masked = np.ma.masked_where(cond2, mesh_bathy)

Load forcing file:

In [ ]:
# Created from notebook: /forcing/sediment-resuspension---parameterization.ipynb
tff = nc.Dataset('/ocean/brogalla/GEOTRACES/data/erosion_rate-20211004.nc')
erosion_rate = np.array(tff.variables['er_rate'])
log_erosion_rate = np.log10(erosion_rate)

#### Figures:

In [ ]:
fig, ax1, proj1= pickle.load(open('/ocean/brogalla/GEOTRACES/pickles/mn-reference.pickle','rb'))

# Sub-domain map: ---------------------------------------------------------------------------
x_sub, y_sub  = proj1(mesh_lon, mesh_lat) 

# tidal stress:
CB = proj1.contourf(x_sub, y_sub, log_erosion_rate, vmin=-10, vmax=-6.5, \
                    levels=np.arange(-10,-6.5,0.25), extend='both')

cbaxes = fig.add_axes([0.13, 0.20, 0.03, 0.25])
CBar = plt.colorbar(CB, cax=cbaxes, ticks=np.arange(-10,-6.5,0.5))
CBar.ax.tick_params(labelsize=5.5)
CBar.set_label('Log. of erosion rate', fontsize=5.5)

fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202204/S5-erosion-rate.jpg', bbox_inches='tight', dpi=300)
fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202204/S5-erosion-rate.svg', bbox_inches='tight', dpi=300, \
            format='svg')

In [ ]:
print('Units [g/m2/yr]') #kg/m2/s --> g/m2/s --> g/m2/year
print('Average erosion rate in domain: ', np.average(erosion_rate[imin:imax, jmin:jmax])*1000*365*24*3600)
print('Maximum erosion rate in domain: ', np.amax(erosion_rate[imin:imax, jmin:jmax])*1000*365*24*3600)
print('Minimum erosion rate in domain: ', np.amin(erosion_rate[imin:imax, jmin:jmax])*1000*365*24*3600)

### Solubility figure:

In [ ]:
gamma   = 0.065
alpha_0  = 0.65
alpha_0_C = 0.3e-6*9*0.8*0.055/gamma # alpha_0 times tuning constant
resus_cst = 0.75

tide           = np.linspace(0,1,100)
changing_solubility_resus_rate = resus_cst*alpha_0_C*gamma*(1-np.exp(-(tide)**2/gamma))*3600*24*365*1e3
constant_solubility_resus_rate = resus_cst*alpha_0_C*(tide**2)*3600*24*365*1e3

In [ ]:
print(f'total constant is {alpha_0_C*gamma*resus_cst}, which is C*alpha_0*gamma')
print(f'so, C*gamma is {alpha_0_C*gamma*resus_cst/alpha_0:.2e} with alpha_0 is 0.65')
print(f'gamma is {gamma}, so C is {alpha_0_C*resus_cst/alpha_0:.2e}')

C=alpha_0_C*resus_cst/alpha_0

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(4.5, 3.74), dpi=300)

ax.plot(tide, changing_solubility_resus_rate, '-k', label=r'$\alpha = \alpha_0\frac{1-e^{-v^2}}{v^2}$')
ax.plot(tide, constant_solubility_resus_rate, '--k', label=r'$\alpha = \alpha_0$')
ax.set_xlim([0, 0.3])
ax.set_ylim([0, 4e3])
ax.set_xlabel('Baratropic tidal speed [m/s]', fontsize=6)
ax.set_ylabel('Sediment resuspension rate [g/m2/yr]', fontsize=6)
ax.tick_params(axis='both', which='major', labelsize=6)

ax.legend(fontsize=6, frameon=False)
fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202204/S6-tail-off.jpg', bbox_inches='tight', dpi=300)
fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202204/S6-tail-off.svg', bbox_inches='tight', dpi=300, format='svg')